In [41]:
from Supporting_functions import *
from WBNS import WBNS_image
import RedLionfishDeconv as rl

import scipy.ndimage as ndi
from aicsimageio.readers import CziReader
from aicsimageio.writers import OmeTiffWriter
import re

from skimage import measure  

import os
import numpy as np
import tifffile
from tqdm import tqdm

import random

# Set the seed for reproducibility
random.seed(42)

In [42]:
# Input


## Tiff source folders

# Source paths for the tiff images if extractImagesFromCzi== True, the folders will be created
srcpath = r'\\WS3\WS3_Remote_Disk\Current_Segovia_lab\Deconvolution_Dilan\data\Synthetic_images_Jupyter\Raw 3D/'
dirSource = srcpath + 'Membranes'
dirTarget = srcpath + 'Membranes'
dirOut    = r'\\WS3\WS3_Remote_Disk\Current_Segovia_lab\Deconvolution_Dilan\data\Deconvolution test\SelfNet0\Simulated_Jupyter\Membranes/'
psf_path = r'C:/SelfNet/PSF_confocal/PSF_Green_488.tif'   #PSF_Red_568, PSF_Green_488, PSF_far_red_647


resolution_scale = 0.2 # XY resolution / Z resolution, -1 if calculate from images

## Generate image planes for training data 
## Process Target image
processTargetImages = True
# BG subtraction
resolution_px = 0 # FWHM of the PSF: 
resolution_pz = 0
noise_lvl = 2
# deconvolution
padding = 32
Niter = 0
# post processing
sigmaLoG = 0.0
sigmaLoGAddScale = 0.0
sigma = 0.8
# image normalization
thres_scale_target = 2.0 # threhold for simple mask for normalization (original 2.0)
percentiles_target = (30, 99.999)
min_v_target = 0
max_v_target = 65535
# save processed images
dirProcTarget = dirSource + '_deconvRL0/' 

## Process Source image
# image normalization
processSourceImages = True
thres_scale_source = 1.5  # threhold for simple mask for normalization (1.5 original)
percentiles_source = (30, 99.999)
min_v_source = 0
max_v_source = 65535



## Parameters for the patch selection
patch_size = 128
stride = 128
signal_intensity_threshold = 5000  #parameter for selecting image patches containing signals
signal_fraction = 0.01    # min amount of pixel with intenity > signal_intensity_threshold
Max_N_patches = 10000


xy_interval=1
xz_interval=4

# Generate image planes for training data 

In [43]:
## Create folders to export data
outDir = dirOut+"raw_data/"
xy_data = outDir+"xy/"
xy_lr_data = outDir+"xy_lr/"
xz_data = outDir+"xz/"
   
createFolder(dirOut)
createFolder(outDir)
createFolder(xy_data)
createFolder(xy_lr_data)
createFolder(xz_data)

folder: '\\WS3\WS3_Remote_Disk\Current_Segovia_lab\Deconvolution_Dilan\data\Deconvolution test\SelfNet0\Simulated_Jupyter\Membranes/' already exists


## Generate image planes for target and intermediate images

In [44]:
# Get all tif images in the target folder, process and export them
image_names = sorted([f for f in os.listdir(dirTarget) if f.endswith('.tif')])

if processTargetImages == True:
    #Creat output folder
    createFolder(dirProcTarget)
    # Open PSF and Prepare PSF
    psf = tifffile.imread(psf_path)
    psf_f = psf.astype(np.float32)
    psf_norm = psf_f/psf_f.sum()

    
count = 1
for i, image_name in tqdm(enumerate(image_names)):

    start_time = time.time()  # Record the start time 
    print(f"** Processing image : {image_name}")
    
    # Open image and get metadata
    img_path = os.path.join(dirTarget, image_name)   
    img = tifffile.imread(img_path)
    img_shape = img.shape
    [physical_pixel_sizeX,physical_pixel_sizeY,physical_pixel_sizeZ] = read_tiff_voxel_size(img_path)    
    scale = physical_pixel_sizeX / physical_pixel_sizeZ
    

    
    if processTargetImages == True:
        # Make image isotropic
        if abs(1.0-scale) > 1e-4: 
            img = reslice(img,'xy',physical_pixel_sizeX,physical_pixel_sizeZ)
        img = img.astype(np.float32)
        new_img_shape = img.shape   
        new_physical_pixel_sizeZ = img_shape[0] * physical_pixel_sizeZ / new_img_shape[0]
        print(f"     -image dimension from : {img_shape} to {new_img_shape}")
        print(f"     -z-space from : {physical_pixel_sizeZ} to {new_physical_pixel_sizeZ}")

        # Get mask
        mask = get_image_simple_mask(img, 0.0, 1.0, thres_scale_target)  
        mask =  mask.astype(np.int16)
    
        # Deconvolution
        if Niter > 0: 
            # Padding image
            img = np.pad(img, padding, mode='reflect')
            imgSizeGB = img.nbytes / (1024 ** 3)
            print('     -size(GB) : ', imgSizeGB)
            # GPU deconvolution
            res_gpu = rl.doRLDeconvolutionFromNpArrays(img, psf, niter=Niter,resAsUint8=False)
            # Removing padding
            img = res_gpu[padding:-padding, padding:-padding, padding:-padding]

        # Remove noise and BG
        if resolution_px > 0:
            img = WBNS_image(img, resolution_px, noise_lvl)
            if resolution_pz > 0:
                img_xz=np.transpose(img,[1,0,2])
                img_xz = WBNS_image(img_xz, resolution_pz, 0)
                img = np.transpose(img_xz,[1,0,2])
        # LoG filter
        if sigmaLoG > 0 :
            imgBorders = ndi.gaussian_laplace(img, sigmaLoG)
            imgBorders *= -1.0
            #imgBorders[imgBorders < 0] = 0
            imgBorders *= sigmaLoGAddScale
            img += imgBorders
        
        # Smooth
        if sigma > 0:
            img = ndi.gaussian_filter(img, sigma)
        
        # Image Normalization
        if percentiles_target[0] > 0 or percentiles_target[1] < 100:
            low_thres, high_thres0 = getNormalizationThresholds(img, percentiles_target) # low thres in whole image
            low_thres0, high_thres = getNormalizationThresholds(img * mask, percentiles_target) # high thres in FG
            img = remove_outliers_image(img, low_thres, high_thres)

        img = image_scaling(img, min_v_target, max_v_target, True)
        img = img.astype(np.uint16)
        
        # Save processed image
        img_out_name = os.path.join(dirProcTarget, image_name)            
        tifffile.imwrite(      
            img_out_name,
            img,
            imagej=True, 
            resolution=(1.0/physical_pixel_sizeX, 1.0/physical_pixel_sizeY), 
            metadata={'spacing': new_physical_pixel_sizeZ, 'unit': 'um', 'axes': 'ZYX'})
        
    # Generate intermediate images down sample and then upsample
    if resolution_scale < 0:
        resolution_scale = scale
        
    img_lr = np.zeros_like(img)
    z,y,x = img.shape    
    new_y = round(y * resolution_scale)
    new_x = round(x * resolution_scale)
    
    for i in range(z):
        temp_img  = cv2.resize(img[i,:,:],(new_x,new_y),interpolation=cv2.INTER_CUBIC)
        #img_lr[i,:,:] = cv2.resize(temp_img,(x,y),interpolation=cv2.INTER_CUBIC)  
        img_lr[i,:,:] = cv2.resize(temp_img,(x,y),interpolation=cv2.INTER_LINEAR) 
            
    # Export planes for target and intermediate images, each plane as a TIFF image 
    
    for i in range(z):
        outName_target = f"{xy_data}{count}.tif"
        outName_interm = f"{xy_lr_data}{count}.tif"
        tifffile.imwrite(outName_target, img[i,:,:])
        tifffile.imwrite(outName_interm, img_lr[i,:,:])      
        count += 1
  
    Elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {image_name}, {count-1} images exported ") 

0it [00:00, ?it/s]

** Processing image : Membrane_GT_Ch2_15_SNR.tif
     -image dimension from : (272, 884, 1696) to (272, 884, 1696)
     -z-space from : 1.0 to 1.0
mode_result.mode :  109.0
     -threshold_value: 218.0
Intensity Norm  from (0 , 909) to  (0, 65535)  



1it [01:16, 76.74s/it]

Elapsed Time: 76.7365 seconds, image Membrane_GT_Ch2_15_SNR.tif, 272 images exported 
** Processing image : Membrane_GT_Ch2_1_SNR.tif
     -image dimension from : (272, 884, 1696) to (272, 884, 1696)
     -z-space from : 1.0 to 1.0
mode_result.mode :  169.0
     -threshold_value: 338.0
Intensity Norm  from (0 , 875) to  (0, 65535)  



2it [02:32, 76.16s/it]

...

Elapsed Time: 76.2514 seconds, image Membrane_GT_G2_15_SNR.tif, 2698 images exported 
** Processing image : Membrane_GT_G2_1_SNR.tif
     -image dimension from : (280, 864, 1654) to (280, 864, 1654)
     -z-space from : 1.0 to 1.0
mode_result.mode :  170.0
     -threshold_value: 340.0
Intensity Norm  from (0 , 899) to  (0, 65535)  



11it [13:59, 74.27s/it]

Elapsed Time: 72.9481 seconds, image Membrane_GT_G2_1_SNR.tif, 2978 images exported 
** Processing image : Membrane_GT_G2_5_SNR.tif
     -image dimension from : (280, 864, 1654) to (280, 864, 1654)
     -z-space from : 1.0 to 1.0
mode_result.mode :  129.0
     -threshold_value: 258.0
Intensity Norm  from (0 , 918) to  (0, 65535)  



12it [15:17, 76.48s/it]

Elapsed Time: 78.3222 seconds, image Membrane_GT_G2_5_SNR.tif, 3258 images exported 


## Generate image planes for source images

In [45]:
# Get all tif images in the dirSource folder, process and export them
image_names = sorted([f for f in os.listdir(dirSource) if f.endswith('.tif')])

count = 1
for i, image_name in tqdm(enumerate(image_names)):

    start_time = time.time()  # Record the start time 
    print(f"** Processing image : {image_name}")
    
    # Open image and get metadata
    img_path = os.path.join(dirSource, image_name)   
    img = tifffile.imread(img_path)
    img_shape = img.shape
    [physical_pixel_sizeX,physical_pixel_sizeY,physical_pixel_sizeZ] = read_tiff_voxel_size(img_path)    
    scale = physical_pixel_sizeX / physical_pixel_sizeZ
 

    if processSourceImages: 
        # Get mask
        mask = get_image_simple_mask(img, 0.0, 1.0, thres_scale_source)  
        mask =  mask.astype(np.int16)
        # Image Normalization
        if percentiles_source[0] > 0 or percentiles_source[1] < 100:
            low_thres, high_thres0 = getNormalizationThresholds(img, percentiles_source) # low thres in whole image
            low_thres0, high_thres = getNormalizationThresholds(img * mask, percentiles_source) # high thres in FG
            img = remove_outliers_image(img, low_thres, high_thres)

        img = image_scaling(img, min_v_source, max_v_source, True)
        img = img.astype(np.uint16)

    # reslice Image
    img_xz = reslice(img,'xz',physical_pixel_sizeX,physical_pixel_sizeZ)
    
    z,y,x = img_xz.shape 
    
    for i in range(z):
        outName_target = f"{xz_data}{count}.tif"
        tifffile.imwrite(outName_target, img_xz[i,:,:])
        count += 1
  
    # reslice Image
    img_yz = reslice(img,'yz',physical_pixel_sizeX,physical_pixel_sizeZ)
    
    z,y,x = img_yz.shape 
    
    for i in range(z):
        outName_target = f"{xz_data}{count}.tif"
        tifffile.imwrite(outName_target, img_yz[i,:,:])
        count += 1

    Elapsed_time = time.time() - start_time
    print(f"Elapsed Time: {Elapsed_time:.4f} seconds, image {image_name}, {count-1} images exported ")    
    

0it [00:00, ?it/s]

** Processing image : Membrane_GT_Ch2_15_SNR.tif
mode_result.mode :  109
     -threshold_value: 163.5
Intensity Norm  from (0 , 962) to  (0, 65535)  



1it [01:33, 93.57s/it]

Elapsed Time: 93.5746 seconds, image Membrane_GT_Ch2_15_SNR.tif, 2580 images exported 
** Processing image : Membrane_GT_Ch2_1_SNR.tif
mode_result.mode :  169
     -threshold_value: 253.5
Intensity Norm  from (0 , 2429) to  (0, 65534)  



2it [03:05, 92.59s/it]

...

Elapsed Time: 94.6276 seconds, image Membrane_GT_G2_15_SNR.tif, 25648 images exported 
** Processing image : Membrane_GT_G2_1_SNR.tif
mode_result.mode :  170
     -threshold_value: 255.0
Intensity Norm  from (0 , 2549) to  (0, 65535)  



11it [17:15, 92.99s/it]

Elapsed Time: 92.2576 seconds, image Membrane_GT_G2_1_SNR.tif, 28166 images exported 
** Processing image : Membrane_GT_G2_5_SNR.tif
mode_result.mode :  129
     -threshold_value: 193.5
Intensity Norm  from (0 , 1042) to  (0, 65535)  



12it [18:51, 94.28s/it]

Elapsed Time: 95.7628 seconds, image Membrane_GT_G2_5_SNR.tif, 30684 images exported 


# Generate training data from image planes

In [46]:
# Create folder output 
train_data_path = os.path.join(dirOut, 'train_data/')
createFolder(train_data_path)

In [47]:
def calculate_fraction_over_threshold(crop_xy: np.ndarray, signal_intensity_threshold: float) -> float:
    total_elements = crop_xy.size  # Total number of elements in the array
    elements_above_threshold = np.sum(crop_xy > signal_intensity_threshold)  # Count elements greater than threshold
    percentage = (elements_above_threshold / total_elements) 

    return percentage

# Initialize arrays

xy = []
xy_lr = []
xz = []

# Loop over lateral images
file_list_xy = os.listdir(xy_data)
for i in tqdm(range(0, len(file_list_xy), xy_interval)):
    xy_img = tifffile.imread(xy_data + str(i + 1) + '.tif')
    xy_lr_img = tifffile.imread(xy_lr_data + str(i + 1) + '.tif')
    L0 = min(xy_img.shape[0], xy_lr_img.shape[0])
    L1 = min(xy_img.shape[1], xy_lr_img.shape[1])
    for m in range(0, L0 - patch_size + 1, stride):
        for n in range(0, L1 - patch_size + 1, stride):
            crop_xy    =    xy_img[m:m + patch_size, n:n + patch_size]
            crop_xy_lr = xy_lr_img[m:m + patch_size, n:n + patch_size]
            
            signal_f = calculate_fraction_over_threshold(crop_xy, signal_intensity_threshold)            
            if signal_f > signal_fraction:
                xy.append(crop_xy)
                xy_lr.append(crop_xy_lr)

# Loop over axial images   
file_list_xz = os.listdir(xz_data)
for i in tqdm(range(0, len(file_list_xz), xz_interval)):
    xz_img = tifffile.imread(xz_data + str(i + 1) + '.tif')
    for m in range(0, xz_img.shape[0] - patch_size + 1, stride):
        for n in range(0, xz_img.shape[1] - patch_size + 1, stride):
            crop_xz = xz_img[m:m + patch_size, n:n + patch_size]
            signal_f = calculate_fraction_over_threshold(crop_xz, signal_intensity_threshold)
            if signal_f > signal_fraction:
                xz.append(crop_xz)# Chose randomly the patches
minPatches = np.amin([len(xy),len(xz) ])

if Max_N_patches > minPatches:
    Max_N_patches = minPatches 
    
# Random index for xy
random_selection_indices = random.sample(range(len(xy)), Max_N_patches)  # Get the random indices

# Use the same indices to select elements from both lists
xy_new = [xy[i] for i in random_selection_indices]
xy_lr_new = [xy_lr[i] for i in random_selection_indices]

# Random elemnts form xz

xz_new = random.sample(xz, Max_N_patches)

print(f" From xy: {len(xy)}, xy_lr: {len(xy_lr)}, xz: {len(xz)} to xy: {len(xy_new)}, xy_lr: {len(xy_lr_new)}, xz: {len(xz_new)}")


100%|██████████████████████████████████████████████████████████████████████████████| 7671/7671 [01:43<00:00, 73.78it/s]

 From xy: 243531, xy_lr: 243531, xz: 125849 to xy: 10000, xy_lr: 10000, xz: 10000


In [48]:
# Chose randomly the patches
minPatches = np.amin([len(xy),len(xz) ])

if Max_N_patches > minPatches:
    Max_N_patches = minPatches 
    
# Random index for xy
random_selection_indices = random.sample(range(len(xy)), Max_N_patches)  # Get the random indices

# Use the same indices to select elements from both lists
xy_new = [xy[i] for i in random_selection_indices]
xy_lr_new = [xy_lr[i] for i in random_selection_indices]

# Random elemnts form xz

xz_new = random.sample(xz, Max_N_patches)

print(f" From xy: {len(xy)}, xy_lr: {len(xy_lr)}, xz: {len(xz)} to xy: {len(xy_new)}, xy_lr: {len(xy_lr_new)}, xz: {len(xz_new)}")


 From xy: 243531, xy_lr: 243531, xz: 125849 to xy: 10000, xy_lr: 10000, xz: 10000


In [49]:
# Convert to arays and save

xy = np.array(xy_new, dtype=np.float32)
xy_lr = np.array(xy_lr_new, dtype=np.float32)
xz = np.array(xz_new, dtype=np.float32)
print(xy.shape, xy_lr.shape, xz.shape)

np.savez(os.path.join(train_data_path, 'train_data.npz'), xy=xy, xy_lr=xy_lr, xz=xz)

(10000, 128, 128) (10000, 128, 128) (10000, 128, 128)


In [50]:
#Save tiff to double check

import tifffile as tiff
tiff.imwrite(os.path.join(train_data_path,'xy.tif'), xy)
tiff.imwrite(os.path.join(train_data_path,'xy_lr.tif'), xy_lr)
tiff.imwrite(os.path.join(train_data_path,'xz.tif'), xz)